In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [19]:
from pydantic import BaseModel
from datetime import datetime
from typing import Tuple

# Defining Schema(model)
class Delievery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int,int]


d = Delievery(timestamp='2020-01-02T03:04:05Z',dimensions=[654,23])

print(d.timestamp)
print(repr(d.timestamp))
print(d.dimensions)

2020-01-02 03:04:05+00:00
datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(654, 23)


In [4]:
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 53,
    'signup_ts': '2020-01-02T03:04:05Z',
    'tastes': {
        'chicken': 45
    }
}

user = User(**external_data)

print(type(user))
print(user.id)
print(user.model_dump()) # convert into json

<class '__main__.User'>
53
{'id': 53, 'name': 'John Doe', 'signup_ts': datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC)), 'tastes': {'chicken': 45}}


#### Just for understanding

In [5]:
external_data = {
    'id': 53,
    'signup_ts': '2020-01-02T03:04:05Z',
    'tastes': {
        'chicken': 45
    }
}

def func(id,signup_ts,tastes):
    print(id)
    print(signup_ts)
    print(tastes)


# printing multiple variables giving dictionary
func(**external_data)


# def func(**kwargs):
#     print(kwargs)


## printing Dictionary giving unlimited arguments
# func(key='fjdskl',value='fjdkls')

53
2020-01-02T03:04:05Z
{'chicken': 45}


In [11]:
from pydantic import ValidationError

# class User(BaseModel):
#     id: int
#     name: str = "Talha Bhai",
#     signup_ts : datetime | None
#     tastes : dict[str,PositiveInt]

class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


    external_data = {
        'id': 64, 
        'name': 'jhkhkj',
        'signup_ts': '2020-01-02T03:04:05Z',
        'tastes': {
            'chicken': 4
        }
    }

    try:
        User(**external_data)
    except ValidationError as e:
        print(e.errors())



PydanticUserError: A non-annotated attribute was detected: `external_data = {'id': 64, 'name': 'jhkhkj', 'signup_ts': '2020-01-02T03:04:05Z', 'tastes': {'chicken': 4}}`. All model fields require a type annotation; if `external_data` is not meant to be a field, you may be able to resolve this error by annotating it as a `ClassVar` or updating `model_config['ignored_types']`.

For further information visit https://errors.pydantic.dev/2.5/u/model-field-missing-annotation

In [4]:
# continuing the above example...

from pydantic import ValidationError,BaseModel,PositiveInt
from datetime import datetime


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


# external_data = {'id': 'not an int', 'tastes': {}}  
external_data = {'id': 54, 'tastes': {}}  

try:
    User(**external_data)  
except ValidationError as e:
    print(e.errors())
    # """
    # [
    #     {
    #         'type': 'int_parsing',
    #         'loc': ('id',),
    #         'msg': 'Input should be a valid integer, unable to parse string as an integer',
    #         'input': 'not an int',
    #         'url': 'https://errors.pydantic.dev/2/v/int_parsing',
    #     },
    #     {
    #         'type': 'missing',
    #         'loc': ('signup_ts',),
    #         'msg': 'Field required',
    #         'input': {'id': 'not an int', 'tastes': {}},
    #         'url': 'https://errors.pydantic.dev/2/v/missing',
    #     },
    # ]
    # """


[{'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 54, 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]


In [11]:
%pip install pydantic[email]

  Obtaining dependency information for email-validator>=2.0.0 from https://files.pythonhosted.org/packages/09/68/d237a603d524ba052e292d71c89939bfa70e3ec7963b255cc3ef7a8770a0/email_validator-2.1.0.post1-py3-none-any.whl.metadata
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   -------- ------------------------------- 61.4/300.4 kB 1.6 MB/s eta 0:00:01
   --------------- ------------------------ 112.6/300.4 kB 1.3 MB/s eta 0:00:01
   ----------------------- ---------------- 174.1/300.4 kB 1.3 MB/s eta 0:00:01
   ------------------------------- -------- 235.5/300.4 kB 1.3 MB/s eta 0:00:01
   -------------------------------------- - 286.7/300.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 300.4/300.4 kB 1.2 MB/s eta 0:00:00
Note: you ma

### Custom Validation

In [27]:
from pydantic import BaseModel, EmailStr, field_validator

class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int

    @field_validator('account_id')
    def validator_account_id(cls,value:int):
        print('cls checking',cls)
        if(value<0):
            # raise ValueError('Negative ID not acceptable')
            print('Negative ID not acceptable')
        
        return value

user = User(name='fjdksl',email='talha@gmail.com',account_id=-53)

user.model_dump_json()

cls checking <class '__main__.User'>
Negative ID not acceptable


'{"name":"fjdksl","email":"talha@gmail.com","account_id":-53}'

### Different Output Types

In [32]:
from pydantic import BaseModel, EmailStr, field_validator

class User(BaseModel):
    name: str
    email: EmailStr
    account_id: int

    @field_validator('account_id')
    def validator_account_id(cls,value:int):
        if(value<0):
            # raise ValueError('Negative ID not acceptable')
            print('Negative ID not acceptable')
        
        return value

user = User(name='fjdksl',email='talha@gmail.com',account_id=65)

print(type(user))
print(user)

<class '__main__.User'>
name='fjdksl' email='talha@gmail.com' account_id=65


#### In Dictionary

In [37]:
user_dict : dict = user.model_dump()
print(type(user_dict))
print(user_dict)

<class 'dict'>
{'name': 'fjdksl', 'email': 'talha@gmail.com', 'account_id': 65}


#### In JSON

In [38]:
usr_json : str = user.model_dump_json()
print(type(usr_json))
print(usr_json)

<class 'str'>
{"name":"fjdksl","email":"talha@gmail.com","account_id":65}


#### Convert Json into Object

In [42]:
user_obj : User = User.model_validate_json(usr_json)
print(type(user_obj))
print(user_obj)
print(user_obj.name)

<class '__main__.User'>
name='fjdksl' email='talha@gmail.com' account_id=65
fjdksl


In [43]:
from datetime import datetime
from typing import Optional

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: list[int] = []

external_data: dict = {'id': '123', 'signup_ts': '2017-06-01 12:22', 'friends': [1, '2', b'3']}
user: User = User(**external_data)
print(user)
print(user.id)

id=123 name='John Doe' signup_ts=datetime.datetime(2017, 6, 1, 12, 22) friends=[1, 2, 3]
123


## Validators

In [16]:
from typing import Any, List

from typing_extensions import Annotated

from pydantic import BaseModel, ValidationError
from pydantic.functional_validators import AfterValidator


def check_squares(v: int) -> int:
    assert v**0.5 % 1 == 0, f'{v} is not a square number'
    return v


def double(v: Any) -> Any:
    return v * 2


MyNumber = Annotated[int, AfterValidator(double), AfterValidator(check_squares)]


class DemoModel(BaseModel):
    number: List[MyNumber]


print(DemoModel(number=[2, 8]))
#> number=[4, 16]
try:
    DemoModel(number=[2, 4])
except ValidationError as e:
    print(e)
    """
    1 validation error for DemoModel
    number.1
      Assertion failed, 8 is not a square number
    assert ((8 ** 0.5) % 1) == 0 [type=assertion_error, input_value=4, input_type=int]
    """

number=[4, 16]
1 validation error for DemoModel
number.1
  Assertion failed, 8 is not a square number [type=assertion_error, input_value=4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/assertion_error


In [17]:
from pydantic import BaseModel, Field

class Blog(BaseModel):
    title: str = Field(...,min_length=5)
    is_active: bool

Blog(title="hi",is_active=False)

ValidationError: 1 validation error for Blog
title
  String should have at least 5 characters [type=string_too_short, input_value='hi', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_short

In [18]:
from pydantic import BaseModel, Field

class Blog(BaseModel):
    title: str = Field(...,min_length=5)
    is_active: bool

Blog(title="hi, world",is_active=False)

Blog(title='hi, world', is_active=False)